<h1><center>SpaGCN Tutorial Easy Mode</center></h1>


<center>Author: Jian Hu,*, Xiangjie Li, Kyle Coleman, Amelia Schroeder, Nan Ma, David J. Irwin, Edward B. Lee, Russell T. Shinohara, Mingyao Li*

### Outline
1. Installation
2. Import modules
3. Read in data
4. Spatial domain detection using SpaGCN
5. Identify SVGs
6. Identify Meta Gene

### 1. Installation
To install SpaGCN package you must make sure that your python version is over 3.5.=. If you don’t know the version of python you can check it by:

In [1]:
import platform
platform.python_version()

'3.11.6'

Note: Because SpaGCN pends on pytorch, you should make sure torch is correctly installed.
<br>
Now you can install the current release of SpaGCN by the following three ways:
#### 1.1 PyPI: Directly install the package from PyPI.

In [2]:
pip3 install SpaGCN
#Note: you need to make sure that the pip is for python3，or we should install SpaGCN by
python3 -m pip install SpaGCN
pip3 install SpaGCN
#If you do not have permission (when you get a permission denied error), you should install SpaGCN by
pip3 install --user SpaGCN

SyntaxError: invalid syntax (432058694.py, line 1)

#### 1.2 Github
Download the package from Github and install it locally:

In [ ]:
git clone https://github.com/jianhuupenn/SpaGCN
cd SpaGCN/SpaGCN_package/
python3 setup.py install --user

#### 1.3 Anaconda
If you do not have Python3.5 or Python3.6 installed, consider installing Anaconda (see Installing Anaconda). After installing Anaconda, you can create a new environment, for example, SpaGCN (you can change to any name you like).

In [ ]:
#create an environment called SpaGCN
conda create -n SpaGCN python=3.7.9
#activate your environment 
conda activate SpaGCN
git clone https://github.com/jianhuupenn/SpaGCN
cd SpaGCN/SpaGCN_package/
python3 setup.py build
python3 setup.py install
conda deactivate

SyntaxError: invalid syntax (1822059814.py, line 2)

### 2. Import python modules

In [1]:
import os,csv,re
import pandas as pd
import numpy as np
import scanpy as sc
import math
from SpaGCN.ez_mode import detect_spatial_domains_ez_mode, spatial_domains_refinement_ez_mode, plot_spatial_domains_ez_mode, detect_meta_genes_ez_mode, detect_SVGs_ez_mode, plot_SVGs_ez_mode, plot_meta_genes_ez_mode
# import SpaGCN as spg
# import sys
# sys.path.append("SpaGCN_package/SpaGCN")
from scipy.sparse import issparse
import random, torch
import warnings
warnings.filterwarnings("ignore")
import matplotlib.colors as clr
import matplotlib.pyplot as plt
# import SpaGCN as spg
#In order to read in image data, we need to install some package. Here we recommend package "opencv"
#install opencv in python
#!pip3 install opencv-python
import cv2

OMP: Info #276: omp_set_nested routine deprecated, please use omp_set_max_active_levels instead.


In [26]:
print(os.getcwd())
# os.chdir('../')
# /Users/masonzhang/Desktop/SpaGCN/SpaGCN_package
# os.chdir('../../tutorial')

/Users/masonzhang/Desktop/SpaGCN/SpaGCN_package


In [ ]:
spg.__version__

### 3. Read in data
The current version of SpaGCN requres three input data: 
<br>
1. The gene expression matrix(n by k): expression_matrix.h5;
<br>
2. Spatial coordinateds of samplespositions.txt;
<br>
3. Histology image(optional): histology.tif, can be tif or png or jepg.
<br>
The gene expreesion data can be stored as an AnnData object. AnnData stores a data matrix .X together with annotations of observations .obs, variables .var and unstructured annotations .uns. 

In [3]:
#Read original 10x_h5 data and save it to h5ad
# from scanpy import read_10x_h5
# adata2 = read_10x_h5("../tutorial/data/151674/151674_raw_feature_bc_matrix.h5")
# spatial=pd.read_csv("../tutorial/data/151674/spatial/tissue_positions_list.csv",sep=",",header=None,na_filter=False,index_col=0) 
# adata2.obs["x1"]=spatial[1]
# adata2.obs["x2"]=spatial[2]
# adata2.obs["x3"]=spatial[3]
# adata2.obs["x4"]=spatial[4]
# adata2.obs["x5"]=spatial[5]
# #Select captured samples
# adata2=adata2[adata2.obs["x1"]==1]
# adata2.var_names=[i.upper() for i in list(adata2.var_names)]
# adata2.var["genename"]=adata2.var.index.astype("str")
# adata2.write_h5ad("../tutorial/data/151674/sample_data.h5ad")
#Read in gene expression and spatial location
adata=sc.read("../tutorial/data/151673/sample_data.h5ad")
adata2=sc.read("../tutorial/data/151674/sample_data.h5ad")
#Read in hitology image
img=cv2.imread("../tutorial/data/151673/histology.tif")
img2=cv2.imread("../tutorial/data/151674/spatial/151674_full_image.tiff")

### 4. Spatial domain detection using SpaGCN

- s: This parameter determines the weight given to histology when calculating Euclidean distance between every two spots. ‘s = 1’ means that the histology pixel intensity value has the same scale variance as the (x,y) coordinates, whereas higher value of ‘s’ indicates higher scale variance, hence, higher weight to histology, when calculating the Euclidean distance. 

- b: This parameter determines the area of each spot when extracting color intensity.
- n_clusters: Number of spatial domains wanted.
- res: Resolution in the initial Louvain's Clustering methods. If the number of clusters is known, we can use the spg.search_res() fnction to search for suitable resolution(optional)

In [4]:
def run_model(adata, img): #Set coordinates
    adata.obs["x_array"]=adata.obs["x2"]
    adata.obs["y_array"]=adata.obs["x3"]
    adata.obs["x_pixel"]=adata.obs["x4"]
    adata.obs["y_pixel"]=adata.obs["x5"]
    x_array=adata.obs["x_array"].tolist()
    y_array=adata.obs["y_array"].tolist()
    x_pixel=adata.obs["x_pixel"].tolist()
    y_pixel=adata.obs["y_pixel"].tolist()
    #Run SpaGCN #b=49
    adata.obs["pred"]= detect_spatial_domains_ez_mode(adata, img, x_array, y_array, x_pixel, y_pixel, n_clusters=7, histology=True, s=1, b=24, p=0.5, r_seed=100, t_seed=100, n_seed=100)
    adata.obs["pred"]=adata.obs["pred"].astype('category')
    #Refine domains (optional)
    #shape="hexagon" for Visium data, "square" for ST data.
    adata.obs["refined_pred"]=spatial_domains_refinement_ez_mode(sample_id=adata.obs.index.tolist(), pred=adata.obs["pred"].tolist(), x_array=x_array, y_array=y_array, shape="hexagon")
    adata.obs["refined_pred"]=adata.obs["refined_pred"].astype('category')
run_model(adata, img)
run_model(adata2, img2) 
# issue is w img2, bc when I use img it works
# print(img.shape)
# print(img2.shape)
# (13332, 13332, 3)  SpaGCN's data
# (600, 600, 3) lowres DeepST's data (2000, 2000, 3) hires. 
# Hires and lowres images not enough, have to use full image which can be found under 'Raw data' on this website: http://spatial.libd.org/spatialLIBD/

hello!
Calculating adj matrix using histology image...
Var of c0,c1,c2 =  56.37902237134122 213.8296960765196 94.30149915618661
Var of x,y,z =  5606737.526317932 4468793.817921193 5606737.526317932
Run 1: l [0.01, 1000], p [0.0, 140.59282153866044]
Run 2: l [0.01, 500.005], p [0.0, 24.66988182067871]
Run 3: l [0.01, 250.0075], p [0.0, 3.638960838317871]
Run 4: l [125.00874999999999, 250.0075], p [0.43257105350494385, 3.638960838317871]
Run 5: l [125.00874999999999, 187.508125], p [0.43257105350494385, 1.5553991794586182]
Run 6: l [125.00874999999999, 156.2584375], p [0.43257105350494385, 0.887833833694458]
Run 7: l [125.00874999999999, 140.63359375], p [0.43257105350494385, 0.6356645822525024]
Run 8: l [125.00874999999999, 132.821171875], p [0.43257105350494385, 0.528247594833374]
Run 9: l [128.91496093749998, 132.821171875], p [0.47897565364837646, 0.528247594833374]
recommended l =  130.86806640625
Start at res =  0.7 step =  0.1
Initializing cluster centers with louvain, resolution 

#### Plot spatial domains

In [5]:
plot_color=["#F56867","#FEB915","#C798EE","#59BE86","#7495D3","#D1D1D1","#6D1A9C","#15821E","#3A84E6","#997273","#787878","#DB4C6C","#9E7A7A","#554236","#AF5F3C","#93796C","#F9BD3F","#DAB370","#877F6C","#268785"]
ax=plot_spatial_domains_ez_mode(adata, domain_name="pred", x_name="y_pixel", y_name="x_pixel", plot_color=plot_color,size=150000/adata.shape[0], show=False, save=True,save_dir="./sample_results/pred4.png")
ax=plot_spatial_domains_ez_mode(adata, domain_name="refined_pred", x_name="y_pixel", y_name="x_pixel", plot_color=plot_color,size=150000/adata.shape[0], show=False, save=True,save_dir="./sample_results/refined_pred4.png")

**Spatial Domains**![](./sample_results/pred4.png) **Refined Spatial Domains**![](./sample_results/refined_pred4.png)

### 5. Identify SVGs

- target: Target domain to identify SVGs.
- min_in_group_fraction: Minium in-group expression fraction.
- min_in_out_group_ratio: Miniumn (in-group expression fraction) / (out-group expression fraction).
- min_fold_change: Miniumn (in-group expression) / (out-group expression).
- r: Radius to detect a spot's neighboring spots.target: Target domain to identify SVGs.

In [ ]:
target=0
#Read in raw data
adata=sc.read("../tutorial/data/151673/results.h5ad")
raw=sc.read("../tutorial/data/151673/sample_data.h5ad")
#Preprocessing
raw.var_names_make_unique()
raw.obs["pred"]=adata.obs["pred"].astype('category')
raw.obs["x_array"]=raw.obs["x2"]
raw.obs["y_array"]=raw.obs["x3"]
raw.obs["x_pixel"]=raw.obs["x4"]
raw.obs["y_pixel"]=raw.obs["x5"]
raw.X=(raw.X.A if issparse(raw.X) else raw.X)
raw.raw=raw
sc.pp.log1p(raw)
#Set filtering criterials
min_in_group_fraction=0.8
min_in_out_group_ratio=1
min_fold_change=1.5
filtered_info=detect_SVGs_ez_mode(raw, target=0, x_name="x_array", y_name="y_array", domain_name="pred", min_in_group_fraction=min_in_group_fraction, min_in_out_group_ratio=min_in_out_group_ratio, min_fold_change=min_fold_change)

Calculateing adj matrix using xy only...
Calculateing adj matrix using xy only...
Calculateing adj matrix using xy only...
Run 1: radius [1.4142135381698608, 16.970561981201172], num_nbr [1.0, 321.149863760218]
Calculateing adj matrix using xy only...
Run 2: radius [1.4142135381698608, 9.192387759685516], num_nbr [1.0, 117.8283378746594]
Calculateing adj matrix using xy only...
Run 3: radius [1.4142135381698608, 5.303300648927689], num_nbr [1.0, 41.85013623978202]
Calculateing adj matrix using xy only...
Run 4: radius [1.4142135381698608, 3.3587570935487747], num_nbr [1.0, 20.04632152588556]
Calculateing adj matrix using xy only...
Run 5: radius [2.386485315859318, 3.3587570935487747], num_nbr [8.7574931880109, 20.04632152588556]
Calculateing adj matrix using xy only...
recommended radius =  2.8726212047040462 num_nbr=12.524523160762943
radius= 2.8726212047040462 average number of neighbors for each spot is 12.524523160762943
 Cluster 0 has neighbors:
Dmain  3 :  863
Dmain  2 :  517
SV

In [ ]:
filtered_info

,genes,in_group_fraction,out_group_fraction,in_out_group_ratio,in_group_mean_exp,out_group_mean_exp,fold_change,pvals_adj,target_dmain,neighbors
0,CAMK2N1,1.000000,0.944964,1.058242,2.333675,1.578288,2.128434,1.656040e-11,0,"[3, 2]"
2,ENC1,0.998638,0.941848,1.060295,2.457791,1.696083,2.141931,1.552131e-03,0,"[3, 2]"
4,GPM6A,0.997275,0.922118,1.081505,2.224006,1.561187,1.940254,8.602227e-03,0,"[3, 2]"
6,ARPP19,0.982289,0.853583,1.150784,1.889256,1.272106,1.853637,4.823349e-02,0,"[3, 2]"
1,HPCAL1,0.851499,0.465213,1.830342,1.141321,0.406338,2.085448,9.706465e-05,0,"[3, 2]"


#### Plot

In [ ]:
color_self = clr.LinearSegmentedColormap.from_list('pink_green', ['#3AB370',"#EAE7CC","#FD1593"], N=256)
plot_SVGs_ez_mode(adata=raw, gene_list=filtered_info["genes"].tolist(), x_name="y_pixel", y_name="x_pixel", plot_color=color_self, size=50, show=False, save=True, save_dir="./sample_results/")

**CAMK2N1**![](./sample_results/CAMK2N1.png) **ENC1**![](./sample_results/ENC1.png) **GPM6A**![](./sample_results/GPM6A.png) **ARPP19**![](c/ARPP19.png) **HPCAL1**![](./sample_results/HPCAL1.png)

### 6. Identify Meta Gene

- target: Target domain to detect meta gene.
- start_gene: Gene to satrt with. Candidate start genes can be found by lowering the filtering criterials.

In [ ]:
target=2
raw.obs["meta"]=detect_meta_genes_ez_mode(raw, target, x_name="x_array", y_name="y_array", domain_name="pred", start_gene="GFAP", use_raw=False)

Add gene:  MGP
Minus gene:  FTH1
Absolute mean change: 0.8913243
Number of non-target spots reduced to: 1888
Meta gene is:  GFAP+MGP-FTH1
Add gene:  MYL9
Minus gene:  MBP
Absolute mean change: 2.175557
Number of non-target spots reduced to: 563
Meta gene is:  GFAP+MGP-FTH1+MYL9-MBP
Add gene:  KRT8
Minus gene:  MT-ATP6
Absolute mean change: 2.8935516
Number of non-target spots reduced to: 111
Meta gene is:  GFAP+MGP-FTH1+MYL9-MBP+KRT8-MT-ATP6
Meta gene: GFAP+MGP-FTH1+MYL9-MBP+KRT8-MT-ATP6


#### Plot

In [ ]:
plot_meta_genes_ez_mode(raw, x_name="y_pixel", y_name="x_pixel", meta_name="meta", plot_color=color_self, size=50, show=False, save=True, save_dir="./")

**start**![](./sample_results/GFAP.png) **meta gene**![](./sample_results/meta_gene.png)